Import packages

In [10]:
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import csv
import time
import sys

Set URL and directory

In [11]:
base_url = 'https://debatepolitics.com/forums/middle-east.77/'
p_dir = 'Downloaded_pages/US-International-Politics-Middle-East/'

pages_url = []
thread_url = {}

for i in range(1,40):
    pages_url.append('page-'+str(i))

Scrape data and load onto dictonary

In [12]:
i = 1
for page in pages_url:
    page = requests.get(base_url + page)
    page_bs = BeautifulSoup(page.content, 'html.parser')
    disc = page_bs.find('div', {'class': 'structItemContainer-group js-threadList'})
    for thread in disc.find_all('div', {'class': 'structItem-title'}):
        url_val = 'https://debatepolitics.com' + thread.find('a').get('href')
        thread_url [thread.get_text()] = url_val
    
    sleep(2)
    print(f'Percentage of scrape complete {int((i/len(pages_url))*100)}%')
    i+=1

Percentage of scrape complete 2%
Percentage of scrape complete 5%
Percentage of scrape complete 7%
Percentage of scrape complete 10%
Percentage of scrape complete 12%
Percentage of scrape complete 15%
Percentage of scrape complete 17%
Percentage of scrape complete 20%
Percentage of scrape complete 23%
Percentage of scrape complete 25%
Percentage of scrape complete 28%
Percentage of scrape complete 30%
Percentage of scrape complete 33%
Percentage of scrape complete 35%
Percentage of scrape complete 38%
Percentage of scrape complete 41%
Percentage of scrape complete 43%
Percentage of scrape complete 46%
Percentage of scrape complete 48%
Percentage of scrape complete 51%
Percentage of scrape complete 53%
Percentage of scrape complete 56%
Percentage of scrape complete 58%
Percentage of scrape complete 61%
Percentage of scrape complete 64%
Percentage of scrape complete 66%
Percentage of scrape complete 69%
Percentage of scrape complete 71%
Percentage of scrape complete 74%
Percentage of scr

Scrape from each thread

In [13]:
len(thread_url)

779

In [14]:
i = 1
for title, url in thread_url.items():
    thread_html = requests.get(url)
    with open (p_dir + str(i) + '.html', 'w+') as f:
        f.write(str(thread_html.content))
    f.close()
    i += 1
    sleep(2)

In [15]:
thread_data = []

for i in range(1, len(thread_url)+1):
    with open (p_dir + str(i) + '.html', 'r') as f:
        dat = BeautifulSoup(f, 'html.parser')
        dat_com  = dat.find('div', {'class' : 'block-container lbContainer'})
        for comment in dat_com.find_all('article', {'class' : 'message message--post js-post js-inlineModContainer'}):
            comment_msg = ' '.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'bbWrapper'}).get_text()))
            author = ' '.join(re.findall('[^\\\\tn]+[a-zA-Z0-9,.!@?&$%]', comment.find('div', {'class' : 'message-userDetails'}).find('a', {'class' : 'username'}).get_text()))
            # date = comment.find('header', {'class' : 'message-attribution message-attribution--split'}).find('a').get_text('u-concealed')
            user_info = {}
            for user_extra in comment.find_all('dl', {'class' : 'pairs pairs--justified'}):
                user_info[user_extra.find('dt').get_text()] = ''.join(re.findall('[^\\\\nt]+[a-zA-Z0-9,.!@?&$%]',user_extra.find('dd').get_text()))
            # print(user_info)
            if 'Gender' in user_info and user_info['Gender'] != 'Undisclosed':
                gender = user_info['Gender']
            else:
                gender = None
            if 'Location' in user_info and user_info['Location'] != 'Undisclosed':
                location = user_info['Location']
            else:
                location = None
            if 'Political Leaning' in user_info and user_info['Political Leaning'] != 'Undisclosed':
                polLean = user_info['Political Leaning']
            else:
                polLean = None
            thread_data.append([author, comment_msg, i, gender, location, polLean])
        
        print(f'completed parsing {i}.html')
    f.close()

completed parsing 1.html
completed parsing 2.html
completed parsing 3.html
completed parsing 4.html
completed parsing 5.html
completed parsing 6.html
completed parsing 7.html
completed parsing 8.html
completed parsing 9.html
completed parsing 10.html
completed parsing 11.html
completed parsing 12.html
completed parsing 13.html
completed parsing 14.html
completed parsing 15.html
completed parsing 16.html
completed parsing 17.html
completed parsing 18.html
completed parsing 19.html
completed parsing 20.html
completed parsing 21.html
completed parsing 22.html
completed parsing 23.html
completed parsing 24.html
completed parsing 25.html
completed parsing 26.html
completed parsing 27.html
completed parsing 28.html
completed parsing 29.html
completed parsing 30.html
completed parsing 31.html
completed parsing 32.html
completed parsing 33.html
completed parsing 34.html
completed parsing 35.html
completed parsing 36.html
completed parsing 37.html
completed parsing 38.html
completed parsing 39.

AttributeError: 'NoneType' object has no attribute 'get_text'

In [16]:
clm_names = ['Poster' , 'Content', 'Thread', 'Gender', 'Location', 'Political_leaning']

df = pd.DataFrame(thread_data, columns = clm_names)

df.head()

,Poster,Content,Thread,Gender,Location,Political_leaning
0,Jacksin PA,Iran Is Laughin g at Trump an d Placin g Hop...,1,Male,None,Progressive
1,rixare4kids,Jacksin PA said Iran Is Laughin g at Trump a...,1,Female,S. California,Slightly Conservative
2,Felis Leo,Jacksin PA said Iran Is Laughin g at Trump a...,1,Male,California,Conservative
3,TypicalRussian,Jacksin PA said Iran Is Laughin g at Trump a...,1,None,None,Independen
4,TypicalRussian,Felis Leo said Good for t he Iran ian Regime ...,1,None,None,Independen


In [17]:
df.to_csv('US_International_Politics_Middle_East.csv')